In [20]:
import binascii
import struct
from unicorn import *
from unicorn.x86_const import *
from capstone import *

In [21]:
encoded_bytes = binascii.unhexlify(''.join(['33', 'e1', 'c4', '99', '11', '06', '81', '16', 'f0', '32', '9f', 'c4', '91', '17', '06', '81', '14', 'f0', '06', '81', '15', 'f1', 'c4', '91', '1a', '06', '81', '1b', 'e2', '06', '81', '18', 'f2', '06', '81', '19', 'f1', '06', '81', '1e', 'f0', 'c4', '99', '1f', 'c4', '91', '1c', '06', '81', '1d', 'e6', '06', '81', '62', 'ef', '06', '81', '63', 'f2', '06', '81', '60', 'e3', 'c4', '99', '61', '06', '81', '66', 'bc', '06', '81', '67', 'e6', '06', '81', '64', 'e8', '06', '81', '65', '9d', '06', '81', '6a', 'f2', 'c4', '99', '6b', '06', '81', '68', 'a9', '06', '81', '69', 'ef', '06', '81', '6e', 'ee', '06', '81', '6f', 'ae', '06', '81', '6c', 'e3', '06', '81', '6d', 'ef', '06', '81', '72', 'e9', '06', '81', '73', '7c']))
checksum_bytes = binascii.unhexlify(''.join(['55', '8b', 'ec', '51', '8b', '55', '0c', 'b9', 'ff', '00', '00', '00', '89', '4d', 'fc', '85', 'd2', '74', '51', '53', '8b', '5d', '08', '56', '57', '6a', '14', '58', '66', '8b', '7d', 'fc', '3b', 'd0', '8b', 'f2', '0f', '47', 'f0', '2b', 'd6', '0f', 'b6', '03', '66', '03', 'f8', '66', '89', '7d', 'fc', '03', '4d', 'fc', '43', '83', 'ee', '01', '75', 'ed', '0f', 'b6', '45', 'fc', '66', 'c1', 'ef', '08', '66', '03', 'c7', '0f', 'b7', 'c0', '89', '45', 'fc', '0f', 'b6', 'c1', '66', 'c1', 'e9', '08', '66', '03', 'c1', '0f', 'b7', 'c8', '6a', '14', '58', '85', 'd2', '75', 'bb', '5f', '5e', '5b', '0f', 'b6', '55', 'fc', '8b', 'c1', 'c1', 'e1', '08', '25', '00', 'ff', '00', '00', '03', 'c1', '66', '8b', '4d', 'fc', '66', 'c1', 'e9', '08', '66', '03', 'd1', '66', '0b', 'c2']))

In [22]:
def get_decoded_bytes(i):
    decoded_bytes = ""
    for byte in encoded_bytes:
        tmp_hex = hex(((byte ^ i) + 0x22) & 0xFF)
        tmp_hex = tmp_hex[2:].zfill(2)
        decoded_bytes += tmp_hex
    return binascii.unhexlify(decoded_bytes)

In [23]:
def get_checksum(decoded_bytes):
    address = 0x000000
    decoded_bytes_addr = 0x10000
    stack_addr = 0x20000

    emu = Uc(UC_ARCH_X86, UC_MODE_32)  #  initializes an x86 emulator in 32-bit mode
    emu.mem_map(address, 2 * 1024 * 1024)  #  allocates 2MiB memory range
    emu.mem_write(address, checksum_bytes) 
    emu.mem_write(decoded_bytes_addr, decoded_bytes)

    emu.reg_write(UC_X86_REG_ESP, stack_addr)
    # [stack_addr, stack_addr + 4] => return address
    # [stack_addr + 4, stack_addr + 8] => decoded_bytes_addr
    # [stack_addr + 8, stack_addr + 9] => 0x79
    emu.mem_write(stack_addr + 4, struct.pack("<I", decoded_bytes_addr))
    emu.mem_write(stack_addr + 8, struct.pack("<B", 0x79))

    emu.emu_start(address, address + len(checksum_bytes))
    checksum = emu.reg_read(UC_X86_REG_AX)

    # emu.hook_add(UC_HOOK_MEM_READ, emu.mem_read)
    # read_value = emu.mem_read(stack_addr, 16)
    # print(read_value)

    return checksum

In [30]:
for i in range(256):
    decoded_bytes = get_decoded_bytes(i)
    checksum = get_checksum(decoded_bytes)

    if hex(checksum) == "0xfb5e":
        print(f"{i}: {hex(checksum)}\n")

        # http://www.capstone-engine.org/lang_python.html
        md = Cs(CS_ARCH_X86, CS_MODE_32)
        for j in md.disasm(decoded_bytes, 0x0):
            print(f"{j.address}\t{j.mnemonic}\t{j.op_str}")

162: 0xfb5e

0	mov	bl, 0x65
2	mov	byte ptr [ebp - 0x2b], bl
5	mov	byte ptr [ebp - 0x2a], 0x74
9	mov	dl, 0x5f
11	mov	byte ptr [ebp - 0x29], dl
14	mov	byte ptr [ebp - 0x28], 0x74
18	mov	byte ptr [ebp - 0x27], 0x75
22	mov	byte ptr [ebp - 0x26], dl
25	mov	byte ptr [ebp - 0x25], 0x62
29	mov	byte ptr [ebp - 0x24], 0x72
33	mov	byte ptr [ebp - 0x23], 0x75
37	mov	byte ptr [ebp - 0x22], 0x74
41	mov	byte ptr [ebp - 0x21], bl
44	mov	byte ptr [ebp - 0x20], dl
47	mov	byte ptr [ebp - 0x1f], 0x66
51	mov	byte ptr [ebp - 0x1e], 0x6f
55	mov	byte ptr [ebp - 0x1d], 0x72
59	mov	byte ptr [ebp - 0x1c], 0x63
63	mov	byte ptr [ebp - 0x1b], bl
66	mov	byte ptr [ebp - 0x1a], 0x40
70	mov	byte ptr [ebp - 0x19], 0x66
74	mov	byte ptr [ebp - 0x18], 0x6c
78	mov	byte ptr [ebp - 0x17], 0x61
82	mov	byte ptr [ebp - 0x16], 0x72
86	mov	byte ptr [ebp - 0x15], bl
89	mov	byte ptr [ebp - 0x14], 0x2d
93	mov	byte ptr [ebp - 0x13], 0x6f
97	mov	byte ptr [ebp - 0x12], 0x6e
101	mov	byte ptr [ebp - 0x11], 0x2e
105	mov	byte ptr [ebp - 0x1

Unicorn: http://www.unicorn-engine.org/docs/

Capstone: http://www.capstone-engine.org/lang_python.html